# Transformer

In [ ]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time

## 编码器

编码器由6个相同的层堆叠成

In [ ]:
class EncoderDecoder(nn.Module):
       """
       A standard Encoder-Decoder architecture. Base for this and many other models
       """
       def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
              super().__init__()
              self.encoder = encoder
              self.decoder = decoder
              self.src_embed = src_embed
              self.tgt_embed = tgt_embed
              self.generator = generator

       def encode(self, src, src_mask):
              return self.encoder(self.src_embed(src), src_mask)

       def decode(self, memory, src_mask, tgt, tgt_mask):
              return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

       def forward(self, src, tgt, src_mask, tgt_mask):
              return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)


class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super().__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

def clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class LayerNorm(nn.Module):
    """Construct a layernorm module"""

    def __init__(self, features, eps=1e-6):
        super().__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean)/(std + self.eps) + self.b_2

class Encoder(nn.Module):
    """Core encoder is a stack of N layers"""
    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class SublayerConnection(nn.Module):
    """残差连接"""
    def __init__(self, size, dropout):
        super().__init__()
        self.norm = LayerNorm(size)
        self.drop = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        """将残差连接应用于任意相同大小的子层"""
        return x + self.drop(sublayer(self.norm(x)))

编码器层的每一层有两个子层，第一个是多头子注意力机制，第二个是全连接前馈网络

In [ ]:
class EncoderLayer(nn.Module):

    def __init__(self, size, self_attn, feed_forward, dropout):
        super().__init__()
        self.attn = self_attn
        self.feed_forward = feed_forward
        self.size = size
        self.sublayer = clones(SublayerConnection(size, dropout), 2)

    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

## Decoder

解码器也由6个相同的层

In [ ]:

class Decoder(nn.Module):

    def __init__(self, layer, N):
        super().__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)


除了每个编码器层中的两个子层外，解码器还插入了一个第三子层，该子层对编码器堆栈的输出执行多头注意力机制。与编码器类似，我们在每个子层周围使用残差连接，然后进行层归一化。

In [ ]:

class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super().__init__()
        self.size = size
        self.attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, lambda x: self.feed_forward(x))

修改了解码器堆栈中的自注意力子层，以防止位置关注到后续位置。这种掩码操作，加上输出嵌入偏移一个位置的事实，确保了对位置
i 的预测只能依赖于位置小于
i 的已知输出。

In [ ]:

def subsequent_mask(size):
    """mask out subsequent positions"""
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.int8)
    return subsequent_mask == 0

注意力函数

In [ ]:

def attention(query, key, value, mask=None, dropout=None):
    """Compute 'Scaled Dot Product Attention'"""
    d_k = query.size(-1)
    scores = torch.matul(query, key.transpose(-2, -1))/math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

多头注意力

In [ ]:

class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        super().__init__()
        assert d_model % h == 0
        # 假设d_v == d_k4
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(dropout)

    def forward(self, query, key, value, mask=None):
        if mask is not None:

            mask = mask.unsqueeze(1)

        nbatches = query.size(0)
        #(batch_size, seq_len, d_model)->(batch_size, seq_len, num_heads, d_k)->(batch_size, num_heads, seq_len, d_k)
        query, key, value = [linear(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
                             for linear, x in zip(self.linears, (query, key, value))]

        x, self.attn = attention(query, key, value, mask=mask, dropout=self.dropout)

        x = x.transpose(1, 2).contiguous().view(nbatches, -1, self.h * self.d_k)
        del query
        del key
        del value

        return self.linears[-1](x)

## 位置前馈网络

$$ FFN(x) = max(0, xW_{1} + b_{1})W_{2} + b_{2} $$

In [ ]:
class PositionwiseFeedForward(nn.Module):

    def __init__(self, d_model=512, d_ff=2048, dropout=0.1):
        super().__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):

        return self.w_2(self.dropout(self.w_1(x).relu()))

嵌入

In [ ]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super().__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return  self.lut(x) * math.sqrt(self.d_model)

位置编码

In [ ]:

class PositionalEncoding(nn.Module):
    """Implement the PE function"""
    def __init__(self, d_model, dropout, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2)* -(math.log(10000) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, x.size(1)]
        return self.dropout(x)


## 完整模型

In [ ]:

def make_model(
    src_vocab, tgt_vocab,


)
